## TO DO

- [ ] Test with multiple queries
- [ ] Youtube transcript
- [x] Check functionality with paragraphs
- [x] Summarizer integrated
- [x] Add URL below eaach heading
- [ ] Save to backend
- [x] Remove unnecessary headings 

# Crawler

In [42]:
import requests
import json

# Replace YOUR_KEY with your Azure Cognitive Services API key
API_KEY = 'b42a0d68f0c949f582f44edddd782914'

# Set the API endpoint, query parameters, and headers
ENDPOINT = 'https://api.bing.microsoft.com/v7.0/search'
PARAMS = {'q': 'Polygraphy', 'mkt': 'en-US'}
HEADERS = {'Ocp-Apim-Subscription-Key': API_KEY}

# Send an HTTP GET request to the Bing Web Search API
response = requests.get(ENDPOINT, headers=HEADERS, params=PARAMS)

# Raise an exception if the request fails
response.raise_for_status()

# Load the JSON response
response_json = response.json()

# Extract the related terms from the response
related_terms = []
if 'relevantTerms' in response_json:
    related_terms = response_json['relevantTerms']

# Print the related terms
print('Related terms:')
for term in related_terms:
    print(term)


Related terms:


In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

# -*- coding: utf-8 -*-

import json
import os
from pprint import pprint
import requests
import bs4

# Add your Bing Search V7 subscription key and endpoint to your environment variables.
subscription_key = "b42a0d68f0c949f582f44edddd782914"
endpoint = "https://api.bing.microsoft.com/v7.0/search/"

# Query term(s) to search for.
query = "Polygraphy"

# Construct a request
mkt = 'en-US'
params = {'q': query, 'mkt': mkt}
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

# Call the API
try:
    response = requests.get(endpoint, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()

    data2 = data["webPages"]["value"]

    length = len(data2)

    dicts = {}

    urls = []

    for i in range(length):
        url = data2[i]["url"]
        name = data2[i]["name"]
        if url.endswith(".pdf"):
            continue
        #print(name + ": " + url)
        urls.append(url)

    print(urls)
except Exception as ex:
    raise ex


['https://www.merriam-webster.com/dictionary/polygraphy', 'https://en.wikipedia.org/wiki/Polygraph', 'https://www.apa.org/topics/cognitive-neuroscience/polygraph', 'https://www.crimemuseum.org/crime-library/criminal-law/what-is-a-polygraph/', 'https://www.polygraph.org/polygraph-frequently-asked-questions', 'https://www.wikihow.com/Pass-a-Polygraph-Test', 'https://docs.nvidia.com/deeplearning/tensorrt/polygraphy/docs/index.html', 'https://www.bigwas.com/2014/10/polygraphy-lie-detection-reviewer.html', 'https://github.com/NVIDIA/TensorRT/issues/2189', 'https://pypi.org/project/polygraphy/']


### URL priority Level Setting

In [2]:
def sort_urls_by_priority(urls, priority_keywords, ignore_keywords):
    # Assign a priority level based on the index of the first priority keyword in the URL
    priority_levels = [len(priority_keywords) + 1 for url in urls]
    url_priority_pairs = []  # list of tuples where each tuple consists of the URL and its priority level
    for i, url in enumerate(urls):
        if not any(keyword in url for keyword in ignore_keywords):
            # only add the URL to the list if it does not contain any of the ignore keywords
            for j, keyword in enumerate(priority_keywords):
                if keyword in url:
                    priority_levels[i] = j
                    break
            url_priority_pairs.append((url, priority_levels[i]))

    # Sort the list of tuples by the priority level (second element in the tuple) in ascending order
    sorted_url_priority_pairs = sorted(url_priority_pairs, key=lambda x: x[1])

    # Extract the URLs from the sorted list of tuples and return them as a new list
    return [pair[0] for pair in sorted_url_priority_pairs]


ignore_keywords = ['dictionary']
priority_keywords = ['wikipedia','geeksforgeeks','tutorialspoint', 'w3schools']
# Sort the URLs according to the specified priority levels
urls = sort_urls_by_priority(urls, priority_keywords, ignore_keywords)

print(urls)

['https://en.wikipedia.org/wiki/Polygraph', 'https://www.apa.org/topics/cognitive-neuroscience/polygraph', 'https://www.crimemuseum.org/crime-library/criminal-law/what-is-a-polygraph/', 'https://www.polygraph.org/polygraph-frequently-asked-questions', 'https://www.wikihow.com/Pass-a-Polygraph-Test', 'https://docs.nvidia.com/deeplearning/tensorrt/polygraphy/docs/index.html', 'https://www.bigwas.com/2014/10/polygraphy-lie-detection-reviewer.html', 'https://github.com/NVIDIA/TensorRT/issues/2189', 'https://pypi.org/project/polygraphy/']


# FINAL HEADINGS TO JSON FROM URL

In [20]:
from bs4 import BeautifulSoup
import re
import requests
from collections import OrderedDict
import json
import pprint

DELIMITER = '\n'

def extract_source(url):
	agent = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
	source=requests.get(url, headers=agent).text
	return source

def extract_data(source, unwanted):
    soup = BeautifulSoup(source, 'lxml')
    headings = soup.find_all(re.compile("^h[1-6]$"))

    headings_dict = OrderedDict()
    hierarchy = [headings_dict]
    current_dict = headings_dict

    # Convert all elements in the unwanted list to lowercase
    unwanted = [x.lower() for x in unwanted]

    for heading in headings:
        level = int(heading.name[1])

        # Move up the hierarchy if needed
        while level < len(hierarchy):
            hierarchy.pop()
            current_dict = hierarchy[-1]

        # Check if the heading text is in the list of unwanted headings
        if heading.text.strip().replace("[edit]", "").lower() not in unwanted:
            # Add the heading to the current position in the hierarchy
            current_dict[heading.text.strip().replace("[edit]", "")] = OrderedDict()
            current_dict = current_dict[heading.text.strip().replace("[edit]", "")]

            # Move down the hierarchy
            hierarchy.append(current_dict)

            # Add the URL to the current position in the hierarchy
            current_dict["url"] = url

            # Initialize an empty list to store the paragraphs
            paragraphs = []
            total_reading_time = 0

            # Set the starting element to the heading element
            current_element = heading

            # Loop until we reach a different heading element or the end of the document
            while True:
                # Get the next element
                next_element = current_element.next_sibling

                # If the next element is a paragraph, add it to the list
                if next_element and (next_element.name == 'p' or next_element.name == 'li' or next_element.name == 'ul' or next_element.name == 'pre'):
                    text = next_element.text.strip().replace("\n", "").replace("\r", "").replace("\t", "")
                    paragraphs.append(text)
                    total_reading_time += calculate_reading_time(text)

                # If the next element is a heading, break the loop
                elif next_element and next_element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                    break

                # If the next element is None, break the loop
                elif next_element is None:
                    break

                # Set the current element to the next element
                current_element = next_element

            if paragraphs:
                # Check if the reading time is greater than 60 seconds
                if total_reading_time > 60:
                    # Convert the reading time to minutes and round to the nearest whole number
                    total_reading_time = round(total_reading_time / 60)
                    current_dict["total_reading_time"] = str(total_reading_time) + " minutes"
                else:
                    current_dict["total_reading_time"] = str(int(total_reading_time)) + " seconds"

                current_dict["paragraphs"] = paragraphs
                #current_dict["reading_time"] = total_reading_time
            

    headings_dict = json.dumps(headings_dict, indent=4)
    return headings_dict

# import textstat library to calculate reading time
import textstat

def calculate_reading_time(text):
    # Use the textstat library to calculate the reading time
    return textstat.reading_time(text)


#print(extract_data(extract_source(urls[1])))

import wikipedia

dictionaries = []

word = query
wikipedia.set_lang("en")

try:
    page = wikipedia.page(word)
    summary = page.summary
    my_dict = {"Definition": {
        "paragraphs":[summary]
        }
    }
    #print(my_dict)
    dictionaries.append(my_dict)
except wikipedia.exceptions.PageError:
    print("Page not found")


unwanted = ["Contents", "See also", "References", "Further reading", "External links", "navigation menu", "personal tools", "namespaces", "views", "navigation", "contribute", "tools", "print/export", "In other projects", "Languages", "Additional menu", "Related Posts:", "Reader Interactions", "Leave a Reply Cancel reply", "For Members", "Member Benefits", "Continue Reading", "Quick Links", "Upcoming Events", "Share", "Using PyPI", "Contributing to PyPI", "About PyPI", "Help", "Source Distribution", "Download files", "Release history\n\nRelease notifications |\n              RSS feed", "Classifiers", "Maintainers", "Meta", "Project links", "Project details", "Project description"]
# d = extract_data(extract_source(urls[0]), unwanted)

# print(d)
for url in urls:
    try:
        d = extract_data(extract_source(url), unwanted)
        dictionaries.append(json.loads(d.strip().replace("\n", "")))
        
    except:
        # An error occurred while trying to connect to the website.
        # Skip to the next iteration of the loop.
        
        print(f'Sorry, an error occurred while trying to connect to the website: {url}.')
        continue
        
#print(dictionaries)  


#json_str = json.dumps(dicts)
#print(json_str[1])



In [22]:
import pprint
#pprint.pprint(dictionaries)
print(dictionaries)

[{'Definition': {'paragraphs': ['"Cooperative Polygraphy" is the fourth episode of the fifth season of Community, and the 88th episode overall in the series. It was written by Alex Rubens and directed by Tristram Shapeero. The episode originally aired on January 16, 2014 on NBC. In the episode, the study group faces a polygraph test as a group as part of Pierce\'s will. As each member is forced to reveal secrets, they begin to fight with each other.\nThe episode received critical acclaim; however, the episode saw a ratings drop, with 3.07 million viewers watching.']}}, {'Polygraph': {'url': 'https://en.wikipedia.org/wiki/Polygraph', 'Testing procedure': {'url': 'https://en.wikipedia.org/wiki/Polygraph', 'total_reading_time': '29 seconds', 'paragraphs': ['The examiner typically begins polygraph test sessions with a pre-test interview to gain some preliminary information which will later be used to develop diagnostic questions. Then the tester will explain how the polygraph is supposed t

## Similarity Funtion

In [6]:
import math
import string
import sys

# function to create acronym
def fxn(stng):

	# add first letter
	oupt = stng[0]
	
	# iterate over string
	for i in range(1, len(stng)):
		if stng[i-1] == ' ':
		
			# add letter next to space
			oupt += stng[i]
			
	# uppercase oupt
	oupt = oupt.upper()
	return oupt

# function to check acronymn
def is_acro(stng1, stng2):
	if (stng1 == fxn(stng2)) or (stng2 == fxn(stng1)):
		return True
	else:
		return False

# reading the text file
# This functio will return a
# list of the lines of text
# in the file.
def read_file(filename):
	
	try:
		with open(filename, 'r') as f:
			data = f.read()
		return data
	
	except IOError:
		print("Error opening or reading input file: ", filename)
		sys.exit()

# splitting the text lines into words
# translation table is a global variable
# mapping upper case to lower case and
# punctuation to spaces
translation_table = str.maketrans(string.punctuation+string.ascii_uppercase,
									" "*len(string.punctuation)+string.ascii_lowercase)
	
# returns a list of the words
# in the file
def get_words_from_line_list(text):
	
	text = text.translate(translation_table)
	word_list = text.split()
	
	return word_list


# counts frequency of each word
# returns a dictionary which maps
# the words to their frequency.
def count_frequency(word_list):
	
	D = {}
	
	for new_word in word_list:
		
		if new_word in D:
			D[new_word] = D[new_word] + 1
			
		else:
			D[new_word] = 1
			
	return D

# returns dictionary of (word, frequency)
# pairs from the previous dictionary.
def word_frequencies_for_file(text):
	
	line_list = text
	word_list = get_words_from_line_list(line_list)
	freq_mapping = count_frequency(word_list)

	""" print("Text", text, ":", )
	print(len(line_list), "lines, ", )
	print(len(word_list), "words, ", )
	print(len(freq_mapping), "distinct words") """

	return freq_mapping


# returns the dot product of two documents
def dotProduct(D1, D2):
	Sum = 0.0
	
	for key in D1:
		
		if key in D2:
			Sum += (D1[key] * D2[key])
			
	return Sum

# returns the angle in radians
# between document vectors
def vector_angle(D1, D2):
	numerator = dotProduct(D1, D2)
	denominator = math.sqrt(dotProduct(D1, D1)*dotProduct(D2, D2))
	
	return math.acos(numerator / denominator)


def documentSimilarity(filename_1, filename_2):
	
# filename_1 = sys.argv[1]
# filename_2 = sys.argv[2]
	sorted_word_list_1 = word_frequencies_for_file(filename_1)
	sorted_word_list_2 = word_frequencies_for_file(filename_2)
	distance = vector_angle(sorted_word_list_1, sorted_word_list_2)
	if is_acro(filename_1, filename_2) == True:
		distance = 0
	
	#print("The distance between the documents is: % 0.6f (radians)"% distance)
	return distance
	
# Driver code
#documentSimilarity('Binary phase-shift keying (BPSK)[edit]', 'Binary Phase Shift Keying')


## Similary strings function

In [7]:
def find_similar_strings(dict1, dict2, processed_strings=set()):
    found_similar = False
    dict2_copy = dict2.copy()
    for key2 in dict2_copy:
        if key2 and key2 != 'paragraphs':
            best_match = None
            best_similarity = 1.0
            for key1 in dict1:
                if key1 and key1 != 'paragraphs' and key1 != 'url' and key1 not in processed_strings:
                    similarity = documentSimilarity(key1, key2)
                    if similarity < 0.7 and similarity < best_similarity:
                        best_match = key1
                        best_similarity = similarity
            if best_match is not None:
                dict1[best_match].update(dict2[key2])
                del dict2[key2]
                print(f'strings {best_match} and {key2} are similar')
                found_similar = True
                processed_strings.add(best_match)
                processed_strings.add(key2)
            elif isinstance(dict2[key2], dict):
                found_similar = find_similar_strings(dict1, dict2[key2], processed_strings)
                if found_similar:
                    break
    for key1 in dict1:
        if not found_similar and isinstance(dict1[key1], dict):
            found_similar = find_similar_strings(dict1[key1], dict2, processed_strings)
            if found_similar:
                break

    return found_similar

In [8]:
result ={}
result = dictionaries[0]

for i in range(1, len(dictionaries)):
    find_similar_strings(result, dictionaries[i])
    result.update(dictionaries[i])

#print(result)

strings View Full Calendar and View Full Calendar are similar


In [88]:
resultN = {}
print(resultN)

{}


In [11]:
import textstat

def calculate_reading_time(d):
    total_reading_time = 0
    for key, value in d.copy().items():
        if key == "paragraphs":
            # Concatenate the list of strings into a single string
            text = ' '.join(value)
            # Calculate reading time for the list of strings
            reading_time = textstat.reading_time(text)
            # Add the reading time to a node with the key 'reading_time'
            total_reading_time += reading_time
        elif type(value) is dict:
            # Recursively call the function for nested dictionaries
            total_reading_time += calculate_reading_time(value)

    # Add the total reading time for all child nodes to the parent node 
    parent_node = d.copy()
    for key, value in parent_node.items():
        if type(value) == dict:
            value["total_reading_time"] = total_reading_time
            break
    #return the total_reading_time
    return total_reading_time


calculate_reading_time(result)

# Edit the calculate_reading_time function to append the total_read_time node at the top of the dictionary right below parent node at any level


434.46000000000004

In [12]:
import pprint
print(result)

{'Definition': {'paragraphs': ['"Cooperative Polygraphy" is the fourth episode of the fifth season of Community, and the 88th episode overall in the series. It was written by Alex Rubens and directed by Tristram Shapeero. The episode originally aired on January 16, 2014 on NBC. In the episode, the study group faces a polygraph test as a group as part of Pierce\'s will. As each member is forced to reveal secrets, they begin to fight with each other.\nThe episode received critical acclaim; however, the episode saw a ratings drop, with 3.07 million viewers watching.'], 'reading_time': 6.54, 'total_reading_time': 434.46000000000004}, 'Polygraph': {'url': 'https://en.wikipedia.org/wiki/Polygraph', 'Testing procedure': {'url': 'https://en.wikipedia.org/wiki/Polygraph', 'paragraphs': ['The examiner typically begins polygraph test sessions with a pre-test interview to gain some preliminary information which will later be used to develop diagnostic questions. Then the tester will explain how th

## Summarizer of paragraphs

In [49]:
import pprint
diction =  {'Countermeasures[edit]': {'paragraphs': ['Several proposed countermeasures designed to pass polygraph tests have been described. There are two major types of countermeasures:  "general state" (intending to alter the physiological or psychological state of the subject during the test), and "specific point" (intending to alter the physiological or psychological state of the subject at specific periods during the examination, either to increase or decrease responses during critical examination periods).[27]', 'General state: asked how he passed the polygraph test, Central Intelligence Agency officer turned KGB mole Aldrich Ames explained that he sought advice from his Soviet handler and received the simple instruction to: "Get a good night\'s sleep, and rest, and go into the test rested and relaxed. Be nice to the polygraph examiner, develop a rapport, and be cooperative and try to maintain your calm".[36]  Additionally, Ames explained, "There\'s no special magic... Confidence is what does it. Confidence and a friendly relationship with the examiner... rapport, where you smile and you make him think that you like him".[37]Specific point: other suggestions for countermeasures include for the subject to mentally record the control and relevant questions as the examiner reviews them before the interrogation begins. During the interrogation the subject is supposed to carefully control their breathing while answering the relevant questions, and to try to artificially increase their heart rate during the control questions, for example by thinking of something scary or exciting, or by pricking themselves with a pointed object concealed somewhere on the body. In this way the results will not show a significant reaction to any of the relevant questions.[38]']}, 'Use[edit]': {'paragraphs': ['Law enforcement agencies and intelligence agencies in the United States are by far the biggest users of polygraph technology.  In the United States alone most federal law enforcement agencies either employ their own polygraph examiners or use the services of examiners employed in other agencies.[39] In 1978 Richard Helms, the eighth Director of Central Intelligence, stated:', 'Susan McCarthy of Salon said in 2000 that "The polygraph is an American phenomenon, with limited use in a few countries, such as Canada, Israel and Japan."[41]']}}

In [50]:
strings = ['Several proposed countermeasures designed to pass polygraph tests have been described. There are two major types of countermeasures:  "general state" (intending to alter the physiological or psychological state of the subject during the test), and "specific point" (intending to alter the physiological or psychological state of the subject at specific periods during the examination, either to increase or decrease responses during critical examination periods).[27]', 'General state: asked how he passed the polygraph test, Central Intelligence Agency officer turned KGB mole Aldrich Ames explained that he sought advice from his Soviet handler and received the simple instruction to: "Get a good night\'s sleep, and rest, and go into the test rested and relaxed. Be nice to the polygraph examiner, develop a rapport, and be cooperative and try to maintain your calm".[36]  Additionally, Ames explained, "There\'s no special magic... Confidence is what does it. Confidence and a friendly relationship with the examiner... rapport, where you smile and you make him think that you like him".[37]Specific point: other suggestions for countermeasures include for the subject to mentally record the control and relevant questions as the examiner reviews them before the interrogation begins. During the interrogation the subject is supposed to carefully control their breathing while answering the relevant questions, and to try to artificially increase their heart rate during the control questions, for example by thinking of something scary or exciting, or by pricking themselves with a pointed object concealed somewhere on the body. In this way the results will not show a significant reaction to any of the relevant questions.[38]']
result = ' '.join(strings)

par = []
par.append(result)

def remove_newlines(strings):
    stripped = [string.strip() for string in strings]
    return stripped

par2 = remove_newlines(par)

key = "503b7fce68494801a92459745a1dcd06"
endpoint = "https://crawler-cognitive.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example method for summarizing text
def sample_extractive_summarization(client, document):
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import (
        TextAnalyticsClient,
        ExtractSummaryAction
    ) 

    

    poller = client.begin_analyze_actions(
        document,
        actions=[
            ExtractSummaryAction()
        ],
    )

    document_results = poller.result()
    for result in document_results:
        extract_summary_result = result[0]  # first document, first result
        if extract_summary_result.is_error:
            print("...Is an error with code '{}' and message '{}'".format(
                extract_summary_result.code, extract_summary_result.message
            ))
        else:
            """ print("\n{}".format(
                " ".join([sentence.text for sentence in extract_summary_result.sentences]))
            ) """
            answer= "".join([sentence.text for sentence in extract_summary_result.sentences])
            return answer.strip()

#sample_extractive_summarization(client, strings)
print(sample_extractive_summarization(client, par2))

Several proposed countermeasures designed to pass polygraph tests have been described.There are two major types of countermeasures:  "general state" (intending to alter the physiological or psychological state of the subject during the test), and "specific point" (intending to alter the physiological or psychological state of the subject at specific periods during the examination, either to increase or decrease responses during critical examination periods).Specific point: other suggestions for countermeasures include for the subject to mentally record the control and relevant questions as the examiner reviews them before the interrogation begins.


In [51]:
def extract_and_summarize(nested_dict, client):
  def traverse(d):
    for key, value in d.items():
      if key == 'paragraphs' and value is not None:
        result = ''.join(value)
        par = []
        par.append(result)
        def remove_newlines(strings):
            stripped = [string.strip() for string in strings]
            return stripped
        par2 = remove_newlines(par)
        # Check for empty or whitespace-only strings
        if par2 and par2[0].strip():
          summarized_value = sample_extractive_summarization(client, par2)
        else:
          summarized_value = ''
        d[key] = summarized_value
      elif isinstance(value, dict):
        traverse(value)
  traverse(nested_dict)
  return nested_dict



In [52]:
output = extract_and_summarize(result, client)
print(output)

AttributeError: 'str' object has no attribute 'items'

# Navigating to relevant links present under websites

# Text Similarity
https://www.geeksforgeeks.org/measuring-the-document-similarity-in-python/

In [ ]:
import math
import string
import sys

# function to create acronym
def fxn(stng):

	# add first letter
	oupt = stng[0]
	
	# iterate over string
	for i in range(1, len(stng)):
		if stng[i-1] == ' ':
		
			# add letter next to space
			oupt += stng[i]
			
	# uppercase oupt
	oupt = oupt.upper()
	return oupt

# function to check acronymn
def is_acro(stng1, stng2):
	if (stng1 == fxn(stng2)) or (stng2 == fxn(stng1)):
		return True
	else:
		return False

# reading the text file
# This functio will return a
# list of the lines of text
# in the file.
def read_file(filename):
	
	try:
		with open(filename, 'r') as f:
			data = f.read()
		return data
	
	except IOError:
		print("Error opening or reading input file: ", filename)
		sys.exit()

# splitting the text lines into words
# translation table is a global variable
# mapping upper case to lower case and
# punctuation to spaces
translation_table = str.maketrans(string.punctuation+string.ascii_uppercase,
									" "*len(string.punctuation)+string.ascii_lowercase)
	
# returns a list of the words
# in the file
def get_words_from_line_list(text):
	
	text = text.translate(translation_table)
	word_list = text.split()
	
	return word_list


# counts frequency of each word
# returns a dictionary which maps
# the words to their frequency.
def count_frequency(word_list):
	
	D = {}
	
	for new_word in word_list:
		
		if new_word in D:
			D[new_word] = D[new_word] + 1
			
		else:
			D[new_word] = 1
			
	return D

# returns dictionary of (word, frequency)
# pairs from the previous dictionary.
def word_frequencies_for_file(text):
	
	line_list = text
	word_list = get_words_from_line_list(line_list)
	freq_mapping = count_frequency(word_list)

	""" print("Text", text, ":", )
	print(len(line_list), "lines, ", )
	print(len(word_list), "words, ", )
	print(len(freq_mapping), "distinct words") """

	return freq_mapping


# returns the dot product of two documents
def dotProduct(D1, D2):
	Sum = 0.0
	
	for key in D1:
		
		if key in D2:
			Sum += (D1[key] * D2[key])
			
	return Sum

# returns the angle in radians
# between document vectors
def vector_angle(D1, D2):
	numerator = dotProduct(D1, D2)
	denominator = math.sqrt(dotProduct(D1, D1)*dotProduct(D2, D2))
	
	return math.acos(numerator / denominator)


def documentSimilarity(filename_1, filename_2):
	
# filename_1 = sys.argv[1]
# filename_2 = sys.argv[2]
	sorted_word_list_1 = word_frequencies_for_file(filename_1)
	sorted_word_list_2 = word_frequencies_for_file(filename_2)
	distance = vector_angle(sorted_word_list_1, sorted_word_list_2)
	if is_acro(filename_1, filename_2) == True:
		distance = 0
	
	#print("The distance between the documents is: % 0.6f (radians)"% distance)
	return distance
	
# Driver code
documentSimilarity('BPSK-Binary  Phase Shift Keying', 'Binary Phase Shift Keying')


0.46364760900080615

## Loop through 2 dictionaries to find similarities

In [ ]:
# start
dict1 = {
    "RF Wireless World": {
        "Home of RF and Wireless Vendors and Resources": {},
        "BPSK-Binary Phase Shift Keying": {
            "BPSK Applications ": {},
            "What is difference between BPSK and QPSK types": {},
            "BPSK MATLAB CODE": {},
            "Modulation types related links": {},
            "RF and Wireless Terminologies": {
                "Share this page": {
                    "Translate this page": {}
                },
                "RF Wireless Tutorials": {},
                "Popular Tutorials": {},
                "Popular terms": {}
            },
            "Probability of error": {},
        }
    }
}

dict2 = {
    "Phase-shift keying": {
        "Contents": {},
        "Introduction[edit]": {},
        "Binary phase-shift keying (BPSK)[edit]": {
            "Implementation[edit]": {},
            "Bit error rate[edit]": {}
        },
        "Quadrature phase-shift keying (QPSK)[edit]": {
            "Implementation[edit]": {},
            "Probability of error[edit]": {},
            "Variants[edit]": {
                "Offset QPSK (OQPSK)[edit]": {},
                "SOQPSK[edit]": {},
                "\u03c0/4-QPSK[edit]": {},
                "DPQPSK[edit]": {}
            }
        },
        "Higher-order PSK[edit]": {
            "Bit error rate[edit]": {},
            "Spectral efficiency[edit]": {}
        },
        "Differential phase-shift keying (DPSK)[edit]": {
            "Differential encoding[edit]": {},
            "Demodulation[edit]": {},
            "Example: Differentially encoded BPSK[edit]": {},
            "Definitions[edit]": {}
        },
        "Applications[edit]": {},
        "Mutual information with additive white Gaussian noise[edit]": {},
        "See also[edit]": {},
        "Notes[edit]": {},
        "References[edit]": {},
        "Navigation menu": {
            "\nPersonal tools\n": {},
            "\nNamespaces\n": {},
            "\nViews\n": {},
            "\nNavigation\n": {},
            "\nContribute\n": {},
            "\nTools\n": {},
            "\nPrint/export\n": {},
            "\nIn other projects\n": {},
            "\nLanguages\n": {}
        }
    }
}

dict3 = {
    "": {
        "Digital Phase Modulation: BPSK, QPSK, DQPSK": {},
        "Join our Engineering Community! Sign-in with:": {
            "Radio Frequency Modulation": {}
        }
    },
    "Digital Phase Modulation: BPSK, QPSK, DQPSK": {
        "Chapter 4 - Radio Frequency Modulation": {
            "Binary Phase Shift Keying": {},
            "QPSK": {},
            "Variants": {
                "Phase Jumps": {},
                "Differential Encoding": {}
            },
            "": {},
            "Summary": {},
            "Related Content": {},
            "You May Also Like": {
                "Understanding IPC Class 2 vs Class 3 Solder Joints": {},
                "Intel VP Aims to Deliver AI Everywhere and to Everyone": {},
                "Precision Signal Chain Technology With Analog Devices": {},
                "Realizing the Future of Fast EV Charging Through CoolSiC\u2122 Based Topology Design": {},
                "Beamformer Solutions Muscle Up for 5G mmWave Duties": {
                    "Products": {
                        "Applications": {},
                        "Content": {},
                        "Who We Are": {},
                        "Connect With Us": {},
                        "More From Our Network": {},
                        "Sign Up": {}
                    }
                }
            }
        },
        "Welcome Back": {}
    }
}



# Remove subarrays
def remove_subarr(arr):
    results = []
    for subarr in arr:
        results.extend(subarr)
    return results


DELIMITER = '\n'

# Convert dictionary to list
def getKeys(D, answer):
    if not D:
        return
    else:
        for k in D:
            answer.append(k.strip().split(DELIMITER))
            getKeys(D[k], answer)
        for i in answer:
            if i == ['']:
                answer.remove(i)

# Store dictionaries in headings
headings = [dict2, dict1, dict3]

# Create number of lists to store dict_to_list values for getKeys() output
answers = [[] for i in range(len(headings))]

# Declare array to store all the headings that recurr
same_heads = []

same_heads_of_parents = []


if __name__ == "__main__":
    for i,j in enumerate(headings):
        getKeys(j, answers[i])
        answers[i] = remove_subarr(answers[i])
        #print(answers[i])

    
    # Iterating through the sublists in the answers array
    for i in range(len(answers)):
        # Compare headings of answers index i with next index i+1
        for j in answers[i]:
            if i<len(answers)-1:
                for k in answers[i+1]:
                    if documentSimilarity(j,k)<0.6:
                        same_heads_of_parents.append(j)
                        same_heads.append(k)
            else: 
                break

print(same_heads_of_parents)
print(same_heads)

""" for i in answers:
    print(i) """



['Binary phase-shift keying (BPSK)[edit]', 'Probability of error[edit]', 'BPSK-Binary Phase Shift Keying']
['BPSK-Binary Phase Shift Keying', 'Probability of error', 'Binary Phase Shift Keying']


' for i in answers:\n    print(i) '

## Remove same headings from dictionaries

In [ ]:
import pprint

same_heads = ['BPSK-Binary Phase Shift Keying', 'Probability of error', 'Binary Phase Shift Keying']

dict1 = {
    "": {
        "Digital Phase Modulation: BPSK, QPSK, DQPSK": {},
        "Join our Engineering Community! Sign-in with:": {
            "Radio Frequency Modulation": {}
        }
    },
    "Digital Phase Modulation: BPSK, QPSK, DQPSK": {
        "Chapter 4 - Radio Frequency Modulation": {
            "Binary Phase Shift Keying": {},
            "QPSK": {},
            "Variants": {
                "Phase Jumps": {},
                "Differential Encoding": {}
            },
            "": {},
            "Summary": {},
            "Related Content": {},
            "You May Also Like": {
                "Understanding IPC Class 2 vs Class 3 Solder Joints": {},
                "Intel VP Aims to Deliver AI Everywhere and to Everyone": {},
                "Precision Signal Chain Technology With Analog Devices": {},
                "Realizing the Future of Fast EV Charging Through CoolSiC\u2122 Based Topology Design": {},
                "Beamformer Solutions Muscle Up for 5G mmWave Duties": {
                    "Products": {
                        "Applications": {},
                        "Content": {},
                        "Who We Are": {},
                        "Connect With Us": {},
                        "More From Our Network": {},
                        "Sign Up": {}
                    }
                }
            }
        },
        "Welcome Back": {}
    }
}

dict2 = {
    "RF Wireless World": {
        "Home of RF and Wireless Vendors and Resources": {},
        "BPSK-Binary Phase Shift Keying": {
            "BPSK Applications": {},
            "What is difference between BPSK and QPSK types": {},
            "BPSK MATLAB CODE": {},
            "Modulation types related links": {},
            "RF and Wireless Terminologies": {
                "Share this page": {
                    "Translate this page": {}
                },
                "RF Wireless Tutorials": {},
                "Popular Tutorials": {},
                "Popular terms": {}
            }
        }
    }
}

dict3 = {
    "Phase-shift keying": {
        "Contents": {},
        "Introduction[edit]": {},
        "Binary phase-shift keying (BPSK)[edit]": {
            "Implementation[edit]": {},
            "Bit error rate[edit]": {}
        },
        "Quadrature phase-shift keying (QPSK)[edit]": {
            "Implementation[edit]": {},
            "Probability of error[edit]": {},
            "Variants[edit]": {
                "Offset QPSK (OQPSK)[edit]": {},
                "SOQPSK[edit]": {},
                "\u03c0/4-QPSK[edit]": {},
                "DPQPSK[edit]": {}
            }
        },
        "Higher-order PSK[edit]": {
            "Bit error rate[edit]": {},
            "Spectral efficiency[edit]": {}
        },
        "Differential phase-shift keying (DPSK)[edit]": {
            "Differential encoding[edit]": {},
            "Demodulation[edit]": {},
            "Example: Differentially encoded BPSK[edit]": {},
            "Definitions[edit]": {}
        },
        "Applications[edit]": {},
        "Mutual information with additive white Gaussian noise[edit]": {},
        "See also[edit]": {},
        "Notes[edit]": {},
        "References[edit]": {},
        "Navigation menu": {
            "Personal tools": {},
            "Namespaces": {},
            "Views": {},
            "Navigation": {},
            "Contribute": {},
            "Tools": {},
            "Print/export": {},
            "In other projects": {},
            "Languages": {}
        }
    }
}

pprint.pprint(dict2)

my_dict = {}

def delete_keys(d, keys_to_delete):
    if isinstance(d, dict):
        for key, value in list(d.items()):  # Make a copy of the items to avoid modification errors
            if key in keys_to_delete:
                my_dict = d
                del d[key]  # Delete the key-value pair from the dictionary
            else:
                delete_keys(value, keys_to_delete)  # Recursively call the function on the value

delete_keys(dict2, same_heads)
pprint.pprint(dict2)
pprint.pprint(my_dict)

{'RF Wireless World': {'BPSK-Binary Phase Shift Keying': {'BPSK Applications': {},
                                                          'BPSK MATLAB CODE': {},
                                                          'Modulation types related links': {},
                                                          'RF and Wireless Terminologies': {'Popular Tutorials': {},
                                                                                            'Popular terms': {},
                                                                                            'RF Wireless Tutorials': {},
                                                                                            'Share this page': {'Translate this page': {}}},
                                                          'What is difference between BPSK and QPSK types': {}},
                       'Home of RF and Wireless Vendors and Resources': {}}}
{'RF Wireless World': {'Home of RF and Wireless Vendors a

## Append the subheadings for same headings inside the same dictionary

In [ ]:
import pprint

same_heads = ['BPSK-Binary Phase Shift Keying', 'Probability of error', 'Binary Phase Shift Keying']
same_heads_of_parents = ['Binary phase-shift keying (BPSK)[edit]', 'Probability of error[edit]', 'BPSK-Binary Phase Shift Keying']

dict1 = {
    "": {
        "Digital Phase Modulation: BPSK, QPSK, DQPSK": {},
        "Join our Engineering Community! Sign-in with:": {
            "Radio Frequency Modulation": {}
        }
    },
    "Digital Phase Modulation: BPSK, QPSK, DQPSK": {
        "Chapter 4 - Radio Frequency Modulation": {
            "Binary Phase Shift Keying": {},
            "QPSK": {},
            "Variants": {
                "Phase Jumps": {},
                "Differential Encoding": {}
            },
            "": {},
            "Summary": {},
            "Related Content": {},
            "You May Also Like": {
                "Understanding IPC Class 2 vs Class 3 Solder Joints": {},
                "Intel VP Aims to Deliver AI Everywhere and to Everyone": {},
                "Precision Signal Chain Technology With Analog Devices": {},
                "Realizing the Future of Fast EV Charging Through CoolSiC\u2122 Based Topology Design": {},
                "Beamformer Solutions Muscle Up for 5G mmWave Duties": {
                    "Products": {
                        "Applications": {},
                        "Content": {},
                        "Who We Are": {},
                        "Connect With Us": {},
                        "More From Our Network": {},
                        "Sign Up": {}
                    }
                }
            }
        },
        "Welcome Back": {}
    }
}

dict2 = {
    "RF Wireless World": {
        "Home of RF and Wireless Vendors and Resources": {},
        "BPSK-Binary Phase Shift Keying": {
            "BPSK Applications": {},
            "What is difference between BPSK and QPSK types": {},
            "BPSK MATLAB CODE": {},
            "Modulation types related links": {},
            "RF and Wireless Terminologies": {
                "Share this page": {
                    "Translate this page": {}
                },
                "RF Wireless Tutorials": {},
                "Popular Tutorials": {},
                "Popular terms": {}
            }
        }
    }
}

dict3 = {
    "Phase-shift keying": {
        "Contents": {},
        "Introduction[edit]": {},
        "Binary phase-shift keying (BPSK)[edit]": {
            "Implementation[edit]": {},
            "Bit error rate[edit]": {}
        },
        "Quadrature phase-shift keying (QPSK)[edit]": {
            "Implementation[edit]": {},
            "Probability of error[edit]": {},
            "Variants[edit]": {
                "Offset QPSK (OQPSK)[edit]": {},
                "SOQPSK[edit]": {},
                "\u03c0/4-QPSK[edit]": {},
                "DPQPSK[edit]": {}
            }
        },
        "Higher-order PSK[edit]": {
            "Bit error rate[edit]": {},
            "Spectral efficiency[edit]": {}
        },
        "Differential phase-shift keying (DPSK)[edit]": {
            "Differential encoding[edit]": {},
            "Demodulation[edit]": {},
            "Example: Differentially encoded BPSK[edit]": {},
            "Definitions[edit]": {}
        },
        "Applications[edit]": {},
        "Mutual information with additive white Gaussian noise[edit]": {},
        "See also[edit]": {},
        "Notes[edit]": {},
        "References[edit]": {},
        "Navigation menu": {
            "Personal tools": {},
            "Namespaces": {},
            "Views": {},
            "Navigation": {},
            "Contribute": {},
            "Tools": {},
            "Print/export": {},
            "In other projects": {},
            "Languages": {}
        }
    }
}

pprint.pprint(dict2)

# Create an empty dictionary to store the desired portion of the original dictionary
new_dict = {}


# Function to traverse the original dictionary and extract the desired key-value pairs
def extract_keys(d, keys):
    for key, value in d.items():
        if key in keys:
            new_dict[key] = value
        elif isinstance(value, dict):
            extract_keys(value, keys)

# Call the extract_keys function to populate the new dictionary
extract_keys(dict1, same_heads)

# Print the new dictionary to verify that it contains the desired portion of the original dictionary
print(new_dict)


def add_dict_to_nested(d, inner_keys, new_dict):
    # Base case: If the value is not a dictionary, return the original dictionary
    if not isinstance(d, dict):
        return d
    
    # Recursive case: If the value is a dictionary, check if any of the inner keys exists in it
    else:
        for k, v in d.items():
            if k in inner_keys:
                # If an inner key is found, update the value with the new dictionary
                d[k] = {**v, **new_dict}
            else:
                # If an inner key is not found, recursively call the function with the value as the input
                d[k] = add_dict_to_nested(v, inner_keys, new_dict)
        return d

# Call the recursive function
updated_dict = add_dict_to_nested(dict2, same_heads_of_parents, new_dict)

pprint.pprint(updated_dict)

{'RF Wireless World': {'BPSK-Binary Phase Shift Keying': {'BPSK Applications': {},
                                                          'BPSK MATLAB CODE': {},
                                                          'Modulation types related links': {},
                                                          'RF and Wireless Terminologies': {'Popular Tutorials': {},
                                                                                            'Popular terms': {},
                                                                                            'RF Wireless Tutorials': {},
                                                                                            'Share this page': {'Translate this page': {}}},
                                                          'What is difference between BPSK and QPSK types': {}},
                       'Home of RF and Wireless Vendors and Resources': {}}}
{'Binary Phase Shift Keying': {}}
{'RF Wireless World': {

## Account for Acronyms in Text Simmilarity

In [ ]:
# function to create acronym
def fxn(stng):

	# add first letter
	oupt = stng[0]
	
	# iterate over string
	for i in range(1, len(stng)):
		if stng[i-1] == ' ':
		
			# add letter next to space
			oupt += stng[i]
			
	# uppercase oupt
	oupt = oupt.upper()
	return oupt

def is_acro(stng1, stng2):
	if (stng1 == fxn(stng2)) or (stng2 == fxn(stng1)):
		return True
	else:
		return False




is_acro("Phase-shift keying", "\u00a0")


False

# TEST BLOCKS

## Extract Headings From URL

In [ ]:
import bs4
import requests

def extract_source(url):
	agent = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
	source=requests.get(url, headers=agent).text
	#print(source)
	return source

#extract_source('https://geekflare.com/web-scraping-in-javascript/')

def extract_data(source):
	soup=bs4.BeautifulSoup(source, 'lxml')
	names=soup.findAll('*')
	for i in names:
		print(i)

	headings = {}
	h1 = {}
	h2 = {}
	h3 = {}
	h4 = {}
	h5 = {}

	# creating a list of all common heading tags
	heading_tags = ["h1", "h2", "h3", "h4", "h5", "p"]

	i = 0
	a = 0
	b=0
	c=0
	d=0
	e=0 

	all_headings = []
	j = 0
	  
	for tags in soup.find_all(heading_tags):
		print(tags)
		# all_headings.append(tags.name)
		# j=j+1
		# #headings[j] = tags.name + ' -> ' + tags.text.strip()

		""" headings[tags.name + "." + str(i)] = tags.text.strip() """
		# print(tags.name + ' -> ' + tags.text.strip())
	return headings
	""" print(h1)
	print(h2)
	print(all_headings) """

	
print("URL 1")
extract_data(extract_source('https://www.rfwireless-world.com/Terminology/BPSK.html'))

URL 1
<h1><a href="https://www.rfwireless-world.com/"><strong>RF Wireless World</strong></a></h1>
<h2>Home of RF and Wireless Vendors and Resources</h2>
<p>One Stop For Your RF and Wireless Need</p>
<h2>BPSK-Binary Phase Shift Keying</h2>
<p>
This page describes Binary Phase Shift Keying technique or <b>BPSK modulation</b> basics.It mentions link to difference between BPSK and QPSK.BPSK matlab code link is also provided.
</p>
<p>
As mentioned BPSK stands for Binary Phase Shift Keying. It is digital modulation technique. <br/><br/>
As mentioned in the constellation diagram binary 1 and binary 0 are represented by different carrier
phases each is 180 degree apart. The simplest BPSK scheme uses two phases to represent the two binary digits and
is known as binary phase-shift keying. The resulting transmitted single for one bit time is <br/><br/>
S (t) =A* cos (2*pi*fc*t)   for binary 1<br/><br/>
S (t) =A *cos (2*pi*fc*t+pi)   for binary 0<br/><br/>
This is regarded as the most robust digit

{}

## Headings to Nested Dictionaries

In [ ]:
from collections import OrderedDict
from bs4 import BeautifulSoup
import re
import json

DELIMITER = '\n'

html = '<h1><a href="https://www.rfwireless-world.com/"><strong>RF Wireless World</strong></a></h1><h2>Home of RF and Wireless Vendors and Resources</h2><p>One Stop For Your RF and Wireless Need</p><h2>BPSK-Binary Phase Shift Keying</h2> <p> This page describes Binary Phase Shift Keying technique or <b>BPSK modulation</b> basics.It mentions link to difference between BPSK and QPSK.BPSK matlab code link is also provided. </p> <p> As mentioned BPSK stands for Binary Phase Shift Keying. It is digital modulation technique. <br/><br/> As mentioned in the constellation diagram binary 1 and binary 0 are represented by different carrier phases each is 180 degree apart. The simplest BPSK scheme uses two phases to represent the two binary digits and is known as binary phase-shift keying. The resulting transmitted single for one bit time is <br/><br/> S (t) =A* cos (2*pi*fc*t)   for binary 1<br/><br/>S (t) =A *cos (2*pi*fc*t+pi)   for binary 0<br/><br/>This is regarded as the most robust digital modulation technique and is used for long distance wireless communication. When CINR is poor base station choose BPSK modulation technique in most of the adaptive modulation technique adopted in cellular communication.<br/><br/>It is less immune to the interference. Typically KMOD is 1 for BPSK in the equation.<br/></p><p> Figure mention the constellation points and encoding rule,which is taken from IEEE standard 802.16-2004 to demonstrate BPSK concept.</p><h3>BPSK Applications </h3><p> The <b>BPSK</b> modulation is a very basic technique used in various wireless standards such as CDMA, WiMAX (16d, 16e), WLAN 11a, 11b, 11g, 11n, Satellite, DVB, Cable modem etc. It is considered to be more robust among all the modulation types due to difference of 180 degree between two constellation points.Hence it can withstand severe amount of channel conditions or channel fading.It is used in OFDM and OFDMA to modulate the pilot subcarriers used for channel estimation and equalization.As we know different channels are used for specific data transmission in cellular systems.The channels used to transmit system related informations which are very essential are modulated using BPSK modulation.</p><h3>What is difference between BPSK and QPSK types</h3><p>Refer <a href="/Terminology/BPSK-vs-QPSK.html">BPSK vs QPSK</a> page which decribes difference Between BPSK and QPSK modulation types.<br/>Refer <a href="/Terminology/QPSK-vs-OQPSK-vs-pi-4QPSK.html">QPSK vs OQPSK vs pi/4QPSK</a>which describes difference between QPSK,OQPSK and pi/4QPSK modulation types.<br/></p><h3>BPSK MATLAB CODE</h3><p>Refer following link which mentions BPSK MATLAB code and provide matlab codes for other modulation types.<br/><a href="https://www.rfwireless-world.com/source-code/MATLAB/BPSK-QPSK-16QAM-64QAM-modulation-matlab-code.html">BPSK QPSK 16QAM 64QAM modulation matlab code</a><br/></p><h3>Modulation types related links</h3><p><a href="/Terminology/What-is-modulation.html">what is modulation</a>  <a href="/Terminology/MSK-GMSK.html">MSK and GMSK modulation</a>  <a href="/Terminology/8-PSK.html">8-PSK modulation</a>  <a href="/Terminology/QPSK.html">QPSK modulation</a>  <a href="/Terminology/QAM.html">QAM modulation</a>  <a href="https://www.rfwireless-world.com/Terminology/C4FM-vs-CQPSK.html">C4FM vs CQPSK</a>  <a href="/Terminology/differential-encoder-vs-differential-decoder.html">Differential Encoder and Decoder</a>  </p><h3>RF and Wireless Terminologies</h3><h5>Share this page</h5><h5>Translate this page</h5><h4>RF Wireless Tutorials</h4><h4>Popular Tutorials</h4><h4>Popular terms</h4><p>©RF Wireless World 2012, RF &amp; Wireless Vendors and Resources, <a href="https://zypopwebtemplates.com/">Free HTML5 Templates</a></p>'

soup = BeautifulSoup(html, "html.parser")

#heading_tags = ["h1", "h2", "h3", "h4", "h5"]

headings = soup.find_all(re.compile("^h[1-6]$"))
#headings = soup.find_all(heading_tags)

headings_dict = OrderedDict()
hierarchy = [headings_dict]
current_dict = headings_dict

result = {}

# initialize an empty dictionary to store the headings and their associated paragraphs
headings_and_paragraphs = {}

# initialize a variable to store the previous heading element at the same or higher level
previous_heading = None


for heading in headings:
    level = int(heading.name[1])
    
    # Move up the hierarchy if needed
    while level < len(hierarchy):
        hierarchy.pop()
        current_dict = hierarchy[-1]
    
    # Add the heading to the current position in the hierarchy
    current_dict[heading.text.strip()] = OrderedDict()
    current_dict = current_dict[heading.text.strip()]


    # Move down the hierarchy
    hierarchy.append(current_dict)

    """ # get the heading text
    heading_text = heading.text
    
    # initialize an empty list to store the paragraphs under this heading
    paragraphs = []
    
    # find the next sibling element (the element immediately following the heading)
    next_sibling = heading.next_sibling
    
    # keep looping until we reach a heading or the end of the document
    while next_sibling and next_sibling.name not in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
        # if the sibling is a paragraph element, add it to the list of paragraphs
        if next_sibling.name == 'p':
            paragraphs.append(next_sibling.text)

        # move on to the next sibling element
        next_sibling = next_sibling.next_sibling
    
    # add the heading and paragraphs to the result dictionary
    result[heading_text] = paragraphs """


    

headings_dict = json.dumps(headings_dict, indent=4)
print(headings_dict)

# print(result)

{
    "RF Wireless World": {
        "Home of RF and Wireless Vendors and Resources": {},
        "BPSK-Binary Phase Shift Keying": {
            "BPSK Applications": {},
            "What is difference between BPSK and QPSK types": {},
            "BPSK MATLAB CODE": {},
            "Modulation types related links": {},
            "RF and Wireless Terminologies": {
                "Share this page": {
                    "Translate this page": {}
                },
                "RF Wireless Tutorials": {},
                "Popular Tutorials": {},
                "Popular terms": {}
            }
        }
    }
}


### Add paragraphs to dictionary also

In [ ]:
from bs4 import BeautifulSoup
import re
import requests
from collections import OrderedDict
import json
import pprint

DELIMITER = '\n'

def extract_source(url):
	agent = {"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"}
	source=requests.get(url, headers=agent).text
	return source


def extract_data(source):
    soup=BeautifulSoup(source, 'lxml')
    headings = soup.find_all(re.compile("^h[1-6]$"))

    headings_dict = OrderedDict()
    hierarchy = [headings_dict]
    current_dict = headings_dict


    for heading in headings:
        level = int(heading.name[1])
        
        # Move up the hierarchy if needed
        while level < len(hierarchy):
            hierarchy.pop()
            current_dict = hierarchy[-1]
        
        # Add the heading to the current position in the hierarchy
        current_dict[heading.text.strip().replace("[edit]", "")] = OrderedDict()
        current_dict = current_dict[heading.text.strip().replace("[edit]", "")]
        
        # Move down the hierarchy
        hierarchy.append(current_dict)

        # Add the URL to the current position in the hierarchy
        current_dict["url"] = url


        # Initialize an empty list to store the paragraphs
        paragraphs = []

        # Set the starting element to the heading element
        current_element = heading

        # Loop until we reach a different heading element or the end of the document
        while True:
            # Get the next element
            next_element = current_element.next_sibling

            # If the next element is a paragraph, add it to the list
            if next_element and (next_element.name == 'p' or next_element.name == 'li' or next_element.name == 'ul' or next_element.name == 'pre'):
                paragraphs.append(next_element.text.strip().replace("\n", "").replace("\r", "").replace("\t", ""))

            # If the next element is a heading, break the loop
            elif next_element and next_element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                break

            # If the next element is None, break the loop
            elif next_element is None:
                break

            # Set the current element to the next element
            current_element = next_element

        if paragraphs:
            current_dict["paragraphs"] = paragraphs


	
    #pprint.pprint(headings_dict)
    headings_dict = json.dumps(headings_dict, indent=4)
    #print(headings_dict)
    return headings_dict

{
    "RF Wireless World": {
        "Home of RF and Wireless Vendors and Resources": {
            "paragraphs": [
                "One Stop For Your RF and Wireless Need"
            ]
        },
        "BPSK-Binary Phase Shift Keying": {
            "paragraphs": [
                " This page describes Binary Phase Shift Keying technique or BPSK modulation basics.It mentions link to difference between BPSK and QPSK.BPSK matlab code link is also provided. ",
                " As mentioned BPSK stands for Binary Phase Shift Keying. It is digital modulation technique.  As mentioned in the constellation diagram binary 1 and binary 0 are represented by different carrier phases each is 180 degree apart. The simplest BPSK scheme uses two phases to represent the two binary digits and is known as binary phase-shift keying. The resulting transmitted single for one bit time is  S (t) =A* cos (2*pi*fc*t)   for binary 1S (t) =A *cos (2*pi*fc*t+pi)   for binary 0This is regarded as the most robu

### Accessing paragraphs under headings

In [ ]:
from bs4 import BeautifulSoup

# Load the HTML document
html_doc = """
<html>
  <body>
    <h1>Heading 1</h1>
    <p>Paragraph 1 under heading 1</p>
    <p>Paragraph 2 under heading 1</p>
    <h2>Subheading 1</h2>
    <p>Paragraph under subheading 1</p>
    <h2>Subheading 2</h2>
    <p>Paragraph under subheading 2</p>
    <h1>Heading 2</h1>
    <p>Paragraph under heading 2</p>
  </body>
</html>
"""

# Create a BeautifulSoup object
soup = BeautifulSoup(html_doc, 'html.parser')

# Find all heading elements
headings = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

# Iterate over the headings
for heading in headings:
  # Print the heading text
  print(heading.text)

  # Initialize an empty list to store the paragraphs
  paragraphs = []

  # Set the starting element to the heading element
  current_element = heading

  # Loop until we reach a different heading element or the end of the document
  while True:
      # Get the next element
      next_element = current_element.next_sibling

      # If the next element is a paragraph, add it to the list
      if next_element and next_element.name == 'p':
          paragraphs.append(next_element.text)

      # If the next element is a heading, break the loop
      elif next_element and next_element.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
          break

      # If the next element is None, break the loop
      elif next_element is None:
          break

      # Set the current element to the next element
      current_element = next_element

  # Print the list of paragraphs
  print(paragraphs)


Heading 1
['Paragraph 1 under heading 1', 'Paragraph 2 under heading 1']
Subheading 1
['Paragraph under subheading 1']
Subheading 2
['Paragraph under subheading 2']
Heading 2
['Paragraph under heading 2']


## Rearrange Dictionaries for merging

In [ ]:
def find_similar_strings(dict1, dict2, processed_strings=set()):
    found_similar = False
    dict2_copy = dict2.copy()
    for key2 in dict2_copy:
        if key2 and key2 != 'paragraphs':
            best_match = None
            best_similarity = 1.0
            for key1 in dict1:
                if key1 and key1 != 'paragraphs' and key1 not in processed_strings:
                    similarity = documentSimilarity(key1, key2)
                    if similarity < 0.7 and similarity < best_similarity:
                        best_match = key1
                        best_similarity = similarity
            if best_match is not None:
                dict1[best_match].update(dict2[key2])
                del dict2[key2]
                print(f'strings {best_match} and {key2} are similar')
                found_similar = True
                processed_strings.add(best_match)
                processed_strings.add(key2)
            elif isinstance(dict2[key2], dict):
                found_similar = find_similar_strings(dict1, dict2[key2], processed_strings)
                if found_similar:
                    break
    for key1 in dict1:
        if not found_similar and isinstance(dict1[key1], dict):
            found_similar = find_similar_strings(dict1[key1], dict2, processed_strings)
            if found_similar:
                break

    return found_similar

find_similar_strings(dict1, dict2)

dict1.update(dict2)

print(dict1)

find_similar_strings(dict1, dict3)

dict1.update(dict3)

print(dict1)

## Dict to List

In [ ]:
DELIMITER = '\n'
# Convert dictionary to list
def getKeys(D, answer):
    if not D:
        return
    else:
        for k in D:
            #k.rstrip('\n')
            answer.append(k.strip().split(DELIMITER))
            getKeys(D[k], answer)

dictA = {"\nProducts\n": {
                        "\nApplications\n": {},
                        "\nContent\n": {},
                        "\nWho We Are\n": {},
                        "\nConnect With Us\n": {},
                        "\nMore From Our Network\n": {},
                        "\nSign Up\n": {}
                    }}

answer = []
if __name__ == "__main__":
    getKeys(dictA, answer)
    print(answer)

[['Products'], ['Applications'], ['Content'], ['Who We Are'], ['Connect With Us'], ['More From Our Network'], ['Sign Up']]


## Comparing Headings of different dictionaries example

In [ ]:
DELIMITER = '\n'

# start
dict1 = {
    "\u00a0": {
        "\nDigital Phase Modulation: BPSK, QPSK, DQPSK\n": {},
        "\nJoin our Engineering Community! Sign-in with:\n": {
            "Radio Frequency Modulation": {}
        }
    },
    "\nDigital Phase Modulation: BPSK, QPSK, DQPSK\n": {
        "\nChapter 4 - Radio Frequency Modulation\n": {
            "Binary Phase Shift Keying": {},
            "QPSK": {},
            "Variants": {
                "Phase Jumps": {},
                "Differential Encoding": {}
            },
            "\u00a0": {},
            "Summary": {},
            "\nRelated Content\n": {},
            "\nYou May Also Like\n": {
                "\nUnderstanding IPC Class 2 vs Class 3 Solder Joints\n": {},
                "\nIntel VP Aims to Deliver AI Everywhere and to Everyone\n": {},
                "\nPrecision Signal Chain Technology With Analog Devices\n": {},
                "\nRealizing the Future of Fast EV Charging Through CoolSiC\u2122 Based Topology Design\n": {},
                "\nBeamformer Solutions Muscle Up for 5G mmWave Duties\n": {
                    "\nProducts\n": {
                        "\nApplications\n": {},
                        "\nContent\n": {},
                        "\nWho We Are\n": {},
                        "\nConnect With Us\n": {},
                        "\nMore From Our Network\n": {},
                        "\nSign Up\n": {}
                    }
                }
            }
        },
        "Welcome Back": {}
    }
}

dict2 = {
    "Phase-shift keying": {
        "Contents": {},
        "Introduction[edit]": {},
        "Binary phase-shift keying (BPSK)[edit]": {
            "Implementation[edit]": {},
            "Bit error rate[edit]": {}
        },
        "Quadrature phase-shift keying (QPSK)[edit]": {
            "Implementation[edit]": {},
            "Probability of error[edit]": {},
            "Variants[edit]": {
                "Offset QPSK (OQPSK)[edit]": {},
                "SOQPSK[edit]": {},
                "\u03c0/4-QPSK[edit]": {},
                "DPQPSK[edit]": {}
            }
        },
        "Higher-order PSK[edit]": {
            "Bit error rate[edit]": {},
            "Spectral efficiency[edit]": {}
        },
        "Differential phase-shift keying (DPSK)[edit]": {
            "Differential encoding[edit]": {},
            "Demodulation[edit]": {},
            "Example: Differentially encoded BPSK[edit]": {},
            "Definitions[edit]": {}
        },
        "Applications[edit]": {},
        "Mutual information with additive white Gaussian noise[edit]": {},
        "See also[edit]": {},
        "Notes[edit]": {},
        "References[edit]": {},
        "Navigation menu": {
            "\nPersonal tools\n": {},
            "\nNamespaces\n": {},
            "\nViews\n": {},
            "\nNavigation\n": {},
            "\nContribute\n": {},
            "\nTools\n": {},
            "\nPrint/export\n": {},
            "\nIn other projects\n": {},
            "\nLanguages\n": {}
        }
    }
}

# Remove subarrays
def remove_subarr(arr):
    results = []
    for subarr in arr:
        results.extend(subarr)
    return results

# Convert dictionary to list
def getKeys(D, answer):
    if not D:
        return
    else:
        for k in D:
            answer.append(k.strip().split(DELIMITER))
            getKeys(D[k], answer)
            #print(k)
        for i in answer:
            if i == ['']:
                answer.remove(i)




# Store dictionaries in headings
headings = [dict1, dict2]


# Declare array to store all the headings that recurr
same_heads = []


if __name__ == "__main__":
    answer1 = []
    answer2 = []
    getKeys(dict1, answer1)
    getKeys(dict2, answer2)
    
    #answer2.remove([''])

    """ for i in answer1:
        if i == ['']:
            answer1.remove(i)
    for j in answer2:
        if j == ['']:
            answer2.remove(j) """

    answer1 = remove_subarr(answer1)
    answer2 = remove_subarr(answer2)

    print(answer1)
    print(answer2)


    for i in answer1:
        print(i)
        for j in answer2:                
                #answer1.remove(i)
                #answer2.remove(j)

            print(documentSimilarity(i,j))
    
    print(same_heads)

    

['Digital Phase Modulation: BPSK, QPSK, DQPSK', 'Join our Engineering Community! Sign-in with:', 'Radio Frequency Modulation', 'Digital Phase Modulation: BPSK, QPSK, DQPSK', 'Chapter 4 - Radio Frequency Modulation', 'Binary Phase Shift Keying', 'QPSK', 'Variants', 'Phase Jumps', 'Differential Encoding', 'Summary', 'Related Content', 'You May Also Like', 'Understanding IPC Class 2 vs Class 3 Solder Joints', 'Intel VP Aims to Deliver AI Everywhere and to Everyone', 'Precision Signal Chain Technology With Analog Devices', 'Realizing the Future of Fast EV Charging Through CoolSiC™ Based Topology Design', 'Beamformer Solutions Muscle Up for 5G mmWave Duties', 'Products', 'Applications', 'Content', 'Who We Are', 'Connect With Us', 'More From Our Network', 'Sign Up', 'Welcome Back']
['Phase-shift keying', 'Contents', 'Introduction[edit]', 'Binary phase-shift keying (BPSK)[edit]', 'Implementation[edit]', 'Bit error rate[edit]', 'Quadrature phase-shift keying (QPSK)[edit]', 'Implementation[edit

## Cut Paste from one dict to another

In [ ]:
# Dictionary to be added
new_dict = {'new_key': 'new_value'}

# List of inner keys to search for in the nested dictionary
inner_keys_to_find = ['key1', 'inner_key2']

# Nested dictionary
nested_dict = {'key1': {'inner_key1': 'value1'}, 
               'key2': {'inner_key2': {'inner_inner_key2': 'value2'}}, 
               'key3': {'inner_key3': 'value3'}}

def add_dict_to_nested(d, inner_keys, new_dict):
    # Base case: If the value is not a dictionary, return the original dictionary
    if not isinstance(d, dict):
        return d
    
    # Recursive case: If the value is a dictionary, check if any of the inner keys exists in it
    else:
        for k, v in d.items():
            if k in inner_keys:
                # If an inner key is found, update the value with the new dictionary
                d[k] = {**v, **new_dict}
            else:
                # If an inner key is not found, recursively call the function with the value as the input
                d[k] = add_dict_to_nested(v, inner_keys, new_dict)
        return d

# Call the recursive function
updated_dict = add_dict_to_nested(nested_dict, inner_keys_to_find, new_dict)

print(updated_dict)


{'key1': {'inner_key1': 'value1', 'new_key': 'new_value'}, 'key2': {'inner_key2': {'inner_inner_key2': 'value2', 'new_key': 'new_value'}}, 'key3': {'inner_key3': 'value3'}}


## Reading text under the heading

In [ ]:
from bs4 import BeautifulSoup
import re

# Read the HTML document
html = """
<html>
  <body>
    <h1>Heading 1</h1>
    <p>Text under heading 1</p>
    <h2>Heading 2</h2>
    <p>Text under heading 2</p>
    <h3>Heading 3</h3>
    <p>Text under heading 3</p>
    <h4>Heading 4</h4>
    <p>Text under heading 4</p>
    <h5>Heading 5</h5>
    <p>Text under heading 5</p>
    <h6>Heading 6</h6>
    <p>Text under heading 6</p>
  </body>
</html>
"""

# Parse the HTML document
soup = BeautifulSoup(html, 'html.parser')

# Find all headings with tag names "h1" through "h6"
headings = soup.find_all(re.compile("^h[1-6]$"))

# Print the text of the paragraphs under each heading
for heading in headings:
    next_element = heading.next_sibling
    while next_element and next_element.name != "h1" and next_element.name != "h2" and next_element.name != "h3" and next_element.name != "h4" and next_element.name != "h5" and next_element.name != "h6":
        if next_element.name == "p":
            print(next_element.text)
        next_element = next_element.next_sibling


Text under heading 1
Text under heading 2
Text under heading 3
Text under heading 4
Text under heading 5
Text under heading 6


## Summarize paragraphs using cognitive api azure

In [ ]:
strings = ["There are three major classes of digital modulation techniques used for transmission of digitally represented data:\n",
                "Amplitude-shift keying (ASK) Frequency-shift keying (FSK) Phase-shift keying (PSK)",
                "All convey data by changing some aspect of a base signal, the carrier wave (usually a sinusoid), in response to a data signal. In the case of PSK, the phase is changed to represent the data signal. There are two fundamental ways of utilizing the phase of a signal in this way:\n",
                "By viewing the phase itself as conveying the information, in which case the demodulator must have a reference signal to compare the received signal's phase against; or\nBy viewing the change in the phase as conveying information\u00a0\u2013  differential schemes, some of which do not need a reference carrier (to a certain extent).",
                "A convenient method to represent PSK schemes is on a constellation diagram. This shows the points in the complex plane where, in this context, the real and imaginary axes are termed the in-phase and quadrature axes respectively due to their 90\u00b0 separation. Such a representation on perpendicular axes lends itself to straightforward implementation. The amplitude of each point along the in-phase axis is used to modulate a cosine (or sine) wave and the amplitude along the quadrature axis to modulate a sine (or cosine) wave.  By convention, in-phase modulates cosine and quadrature modulates sine.\n",
                "In PSK, the constellation points chosen are usually positioned with uniform angular spacing around a circle. This gives maximum phase-separation between adjacent points and thus the best immunity to corruption. They are positioned on a circle so that they can all be transmitted with the same energy. In this way, the moduli of the complex numbers they represent will be the same and thus so will the amplitudes needed for the cosine and sine waves. Two common examples are \"binary phase-shift keying\" (BPSK) which uses two phases, and \"quadrature phase-shift keying\" (QPSK) which uses four phases, although any number of phases may be used. Since the data to be conveyed are usually binary, the PSK scheme is usually designed with the number of constellation points being a power of two.\n"]

result = ' '.join(strings)

par = []
par.append(result)

def remove_newlines(strings):
    stripped = [string.strip() for string in strings]
    return stripped

par2 = remove_newlines(par)

key = "503b7fce68494801a92459745a1dcd06"
endpoint = "https://crawler-cognitive.cognitiveservices.azure.com/"

from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

# Authenticate the client using your key and endpoint 
def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()

# Example method for summarizing text
def sample_extractive_summarization(client, document):
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.textanalytics import (
        TextAnalyticsClient,
        ExtractSummaryAction
    ) 

    

    poller = client.begin_analyze_actions(
        document,
        actions=[
            ExtractSummaryAction()
        ],
    )

    document_results = poller.result()
    for result in document_results:
        extract_summary_result = result[0]  # first document, first result
        if extract_summary_result.is_error:
            print("...Is an error with code '{}' and message '{}'".format(
                extract_summary_result.code, extract_summary_result.message
            ))
        else:
            print("\n{}".format(
                " ".join([sentence.text for sentence in extract_summary_result.sentences]))
            )

#sample_extractive_summarization(client, strings)
sample_extractive_summarization(client, par2)


There are three major classes of digital modulation techniques used for transmission of digitally represented data: Amplitude-shift keying (ASK) Frequency-shift keying (FSK) Phase-shift keying (PSK) All convey data by changing some aspect of a base signal, the carrier wave (usually a sinusoid), in response to a data signal. A convenient method to represent PSK schemes is on a constellation diagram.


## QnA generator
Patil, S. Question Generation using transformers (Version 1.0.0) [Computer software]. https://github.com/patil-suraj/question_generation

https://github.com/AMontgomerie/question_generator

In [ ]:
from questiongenerator import QuestionGenerator
from questiongenerator import print_qa

c:\Users\sukri\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
text = "\nThis page describes Binary Phase Shift Keying technique or BPSK modulation basics.It mentions link to difference between BPSK and QPSK.BPSK matlab code link is also provided.\n\nAs mentioned BPSK stands for Binary Phase Shift Keying. It is digital modulation technique. \nAs mentioned in the constellation diagram binary 1 and binary 0 are represented by different carrier\nphases each is 180 degree apart. The simplest BPSK scheme uses two phases to represent the two binary digits and\nis known as binary phase-shift keying. The resulting transmitted single for one bit time is \nS (t) =A* cos (2*pi*fc*t)   for binary 1\nS (t) =A *cos (2*pi*fc*t+pi)   for binary 0\nThis is regarded as the most robust digital modulation technique and\nis used for long distance wireless communication. When CINR is poor base station choose BPSK modulation technique in most\nof the adaptive modulation technique adopted in cellular communication.\nIt is less immune to the interference.\nTypically KMOD is 1 for BPSK in the equation.\n Figure mention the constellation points and encoding rule,\nwhich is taken from IEEE standard 802.16-2004 to demonstrate BPSK concept.\n"
qg = QuestionGenerator()
qa_list = qg.generate(
    text,
    num_questions=5,
    answer_style='all',
)
print(qa_list)

Generating questions...

Evaluating QA pairs...

[{'question': 'What is the difference between BPSK and QPSK?', 'answer': 'The simplest BPSK scheme uses two phases to represent the two binary digits and is known as binary phase-shift keying.'}, {'question': 'What is the difference between binary phase shift and QPSK?', 'answer': 'As mentioned in the constellation diagram binary 1 and binary 0 are represented by different carrier phases each is 180 degree apart.'}, {'question': 'What is the KMOD for BPSK?', 'answer': 'Typically KMOD is 1 for BPSK in the equation.'}, {'question': 'What is the link to BPSK?', 'answer': 'BPSK matlab code link is also provided.'}, {'question': 'what is a bpsk modulation?', 'answer': [{'answer': '802', 'correct': False}, {'answer': 'Binary Phase Shift Keying', 'correct': True}, {'answer': 'IEEE', 'correct': False}, {'answer': '180', 'correct': False}]}]


## Loop through lists for similar headings comparison

In [ ]:
def compare_strings(list_of_arrays, similarity_threshold):
    # Initialize an empty list to store the similar elements
    similar_elements = []

    # Iterate over the list of arrays
    for i in range(len(list_of_arrays)):
        # Get the current array
        current_array = list_of_arrays[i]

        # Iterate over the elements in the current array
        for j in range(len(current_array)):
            # Get the current element
            current_element = current_array[j]

            # Iterate over the subsequent arrays
            for k in range(i+1, len(list_of_arrays)):
                # Get the subsequent array
                subsequent_array = list_of_arrays[k]

                # Iterate over the elements in the subsequent array
                for l in range(len(subsequent_array)):
                    # Get the element in the subsequent array
                    subsequent_element = subsequent_array[l]

                    # Calculate the similarity between the current element and the subsequent element
                    similarity = documentSimilarity(current_element, subsequent_element)

                    # If the similarity is below the threshold, add the current element to the list of similar elements
                    if similarity < similarity_threshold:
                        similar_elements.append(current_element)

    # Return the list of similar elements
    return similar_elements

list_of_arrays = [['RF Wireless World', 'Home of RF and Wireless Vendors and Resources', 'BPSK-Binary Phase Shift Keying', 'BPSK Applications', 'What is difference between BPSK and QPSK types', 'BPSK MATLAB CODE', 'Modulation types related links', 'RF and Wireless Terminologies', 'Share this page', 'Translate this page', 'RF Wireless Tutorials', 'Popular Tutorials', 'Popular terms'], ['A BPSK Modulator for 2GHz to 12GHz', 'Authors', 'Related Content', 'Products', 'Cookie Consent', 'Modal title'], ['Digital Phase Modulation: BPSK, QPSK, DQPSK', 'Join our Engineering Community! Sign-in with:', 'Radio Frequency Modulation', 'Digital Phase Modulation: BPSK, QPSK, DQPSK', 'Chapter 4 - Radio Frequency Modulation', 'Binary Phase Shift Keying', 'QPSK', 'Variants', 'Phase Jumps', 'Differential Encoding', 'Summary', 'Related Content', 'You May Also Like', 'What is a CPLD?', 'Simplify Your Automotive Reverse Battery Voltage Testing of Electronic Modules', 'DC and Steady-state AC Circuit Analysis Made Easy with Python', 'Researchers Blind and Deceive LiDAR in Autonomous Vehicles', 'Day 5: ITD 2022 Closes Out With Keynote on Supply Chains, Hot Trends, and More', 'Products', 'Applications', 'Content', 'Who We Are', 'Connect With Us', 'More From Our Network', 'Sign Up', 'Welcome Back'], ['Not Acceptable!'], [], ['Phase-shift keying', 'Contents', 'Introduction[edit]', 'Binary phase-shift keying (BPSK)[edit]', 'Implementation[edit]', 'Bit error rate[edit]', 'Quadrature phase-shift keying (QPSK)[edit]', 'Implementation[edit]', 'Probability of error[edit]', 'Variants[edit]', 'Offset QPSK (OQPSK)[edit]', 'SOQPSK[edit]', 'π/4-QPSK[edit]', 'DPQPSK[edit]', 'Higher-order PSK[edit]', 'Bit error rate[edit]', 'Spectral efficiency[edit]', 'Differential phase-shift keying (DPSK)[edit]', 'Differential encoding[edit]', 'Demodulation[edit]', 'Example: Differentially encoded BPSK[edit]', 'Definitions[edit]', 'Applications[edit]', 'Mutual information with additive white Gaussian noise[edit]', 'See also[edit]', 'Notes[edit]', 'References[edit]', 'Navigation menu', 'Personal tools', 'Namespaces', 'Views', 'Navigation', 'Contribute', 'Tools', 'Print/export', 'In other projects', 'Languages'], ['Phase Shift Keying – PSK', 'Principle of BPSK', 'Expression for BPSK', 'BPSK modulation', 'BPSK Demodulation', 'Advantages of Phase shift keying', 'Disadvantages of Phase shift keying', 'Applications of Phase shift keying', 'Related Terms:', 'Reader Interactions', 'Leave a Reply Cancel reply', 'Primary Sidebar', 'Most Searched Terms', 'Trending Terms', 'New Additions', 'Categories'], ['Digital Communication - Phase Shift Keying', 'Digital Marketing Online Training', 'Digital Marketing Online Training', 'How To Start Your Own Digital Marketing Agency', 'How To Start Your Own Digital Marketing Agency', 'Digital Marketing Agency Elite Consultants Masterclass', 'Digital Marketing Agency Elite Consultants Masterclass', 'Binary Phase Shift Keying (BPSK)', 'Quadrature Phase Shift Keying (QPSK)', 'BPSK Modulator', 'BPSK Demodulator', 'Annual Membership', 'Training for a Team'], ['Not Acceptable!']]
similarity_threshold = 0.6
similar_elements = compare_strings(list_of_arrays, similarity_threshold)
print(similar_elements)


(['BPSK-Binary Phase Shift Keying', 'BPSK-Binary Phase Shift Keying', 'BPSK-Binary Phase Shift Keying', 'Related Content', 'Products', 'Binary Phase Shift Keying', 'Binary Phase Shift Keying', 'Not Acceptable!', 'Binary phase-shift keying (BPSK)[edit]', 'Quadrature phase-shift keying (QPSK)[edit]'], ['Binary Phase Shift Keying', 'Binary phase-shift keying (BPSK)[edit]', 'Binary Phase Shift Keying (BPSK)', 'Related Content', 'Products', 'Phase-shift keying', 'Binary Phase Shift Keying (BPSK)', 'Not Acceptable!', 'Binary Phase Shift Keying (BPSK)', 'Quadrature Phase Shift Keying (QPSK)'])


In [ ]:
list_of_dicts = [OrderedDict([('RF Wireless World', OrderedDict([('Home of RF and Wireless Vendors and Resources', OrderedDict()), ('BPSK-Binary Phase Shift Keying', OrderedDict([('BPSK Applications', OrderedDict()), ('What is difference between BPSK and QPSK types', OrderedDict()), ('BPSK MATLAB CODE', OrderedDict()), ('Modulation types related links', OrderedDict()), ('RF and Wireless Terminologies', OrderedDict([('Share this page', OrderedDict([('Translate this page', OrderedDict())])), ('RF Wireless Tutorials', OrderedDict()), ('Popular Tutorials', OrderedDict()), ('Popular terms', OrderedDict())]))]))]))]), OrderedDict([('A BPSK Modulator for 2GHz to 12GHz', OrderedDict([('Authors', OrderedDict()), ('Related Content', OrderedDict()), ('Products', OrderedDict([('Cookie Consent', OrderedDict([('Modal title', OrderedDict())]))]))]))]), OrderedDict([('', OrderedDict([('Digital Phase Modulation: BPSK, QPSK, DQPSK', OrderedDict()), ('Join our Engineering Community! Sign-in with:', OrderedDict([('Radio Frequency Modulation', OrderedDict())]))])), ('Digital Phase Modulation: BPSK, QPSK, DQPSK', OrderedDict([('Chapter 4 - Radio Frequency Modulation', OrderedDict([('Binary Phase Shift Keying', OrderedDict()), ('QPSK', OrderedDict()), ('Variants', OrderedDict([('Phase Jumps', OrderedDict()), ('Differential Encoding', OrderedDict())])), ('', OrderedDict()), ('Summary', OrderedDict()), ('Related Content', OrderedDict()), ('You May Also Like', OrderedDict([('The Challenges Of Aligning Multiple Connector Sets', OrderedDict()), ('How Matter Addresses Interoperability Issues in Smart Home Devices', OrderedDict()), ('RISC-V: The Tech Between Us Podcast', OrderedDict()), ('ADC Nonlinearity—Missing Codes, Monotonicity, and Nonlinearity Effect on SNR', OrderedDict()), ('Blickfeld Unveils First Smart LiDAR Sensor with Built-in Software Stack', OrderedDict([('Products', OrderedDict([('Applications', OrderedDict()), ('Content', OrderedDict()), ('Who We Are', OrderedDict()), ('Connect With Us', OrderedDict()), ('More From Our Network', OrderedDict()), ('Sign Up', OrderedDict())]))]))]))])), ('Welcome Back', OrderedDict())]))]), OrderedDict([('Not Acceptable!', OrderedDict())]), OrderedDict(), OrderedDict([('Phase-shift keying', OrderedDict([('Contents', OrderedDict()), ('Introduction[edit]', OrderedDict()), ('Binary phase-shift keying (BPSK)[edit]', OrderedDict([('Implementation[edit]', OrderedDict()), ('Bit error rate[edit]', OrderedDict())])), ('Quadrature phase-shift keying (QPSK)[edit]', OrderedDict([('Implementation[edit]', OrderedDict()), ('Probability of error[edit]', OrderedDict()), ('Variants[edit]', OrderedDict([('Offset QPSK (OQPSK)[edit]', OrderedDict()), ('SOQPSK[edit]', OrderedDict()), ('π/4-QPSK[edit]', OrderedDict()), ('DPQPSK[edit]', OrderedDict())]))])), ('Higher-order PSK[edit]', OrderedDict([('Bit error rate[edit]', OrderedDict()), ('Spectral efficiency[edit]', OrderedDict())])), ('Differential phase-shift keying (DPSK)[edit]', OrderedDict([('Differential encoding[edit]', OrderedDict()), ('Demodulation[edit]', OrderedDict()), ('Example: Differentially encoded BPSK[edit]', OrderedDict()), ('Definitions[edit]', OrderedDict())])), ('Applications[edit]', OrderedDict()), ('Mutual information with additive white Gaussian noise[edit]', OrderedDict()), ('See also[edit]', OrderedDict()), ('Notes[edit]', OrderedDict()), ('References[edit]', OrderedDict()), ('Navigation menu', OrderedDict([('Personal tools', OrderedDict()), ('Namespaces', OrderedDict()), ('Views', OrderedDict()), ('Navigation', OrderedDict()), ('Contribute', OrderedDict()), ('Tools', OrderedDict()), ('Print/export', OrderedDict()), ('In other projects', OrderedDict()), ('Languages', OrderedDict())]))]))]), OrderedDict([('Phase Shift Keying – PSK', OrderedDict([('Principle of BPSK', OrderedDict([('Expression for BPSK', OrderedDict())])), ('BPSK modulation', OrderedDict()), ('BPSK Demodulation', OrderedDict([('Advantages of Phase shift keying', OrderedDict()), ('Disadvantages of Phase shift keying', OrderedDict()), ('Applications of Phase shift keying', OrderedDict()), ('Related Terms:', OrderedDict())])), ('Reader Interactions', OrderedDict([('Leave a Reply Cancel reply', OrderedDict())])), ('Primary Sidebar', OrderedDict([('Most Searched Terms', OrderedDict()), ('Trending Terms', OrderedDict()), ('New Additions', OrderedDict()), ('Categories', OrderedDict())]))]))]), OrderedDict([('Digital Communication - Phase Shift Keying', OrderedDict([('Digital Marketing Online Training', OrderedDict([('Digital Marketing Online Training', OrderedDict())])), ('How To Start Your Own Digital Marketing Agency', OrderedDict([('How To Start Your Own Digital Marketing Agency', OrderedDict())])), ('Digital Marketing Agency Elite Consultants Masterclass', OrderedDict([('Digital Marketing Agency Elite Consultants Masterclass', OrderedDict()), ('Binary Phase Shift Keying (BPSK)', OrderedDict()), ('Quadrature Phase Shift Keying (QPSK)', OrderedDict())])), ('BPSK Modulator', OrderedDict()), ('BPSK Demodulator', OrderedDict([('Annual Membership', OrderedDict()), ('Training for a Team', OrderedDict())]))]))]), OrderedDict([('Not Acceptable!', OrderedDict())])]


In [ ]:
def compare_dicts(dict_list, similarity_threshold):
    # Initialize an empty list to store the similar keys
    similar_keys = []

    # Make sure the dict_list is not empty
    if not dict_list:
        return similar_keys

    # Iterate over the dictionaries in the list
    for i in range(len(dict_list)):
        # Get the current dictionary
        current_dict = dict_list[i]

        # Iterate over the keys in the current dictionary
        for key1, value1 in current_dict.items():
            # Skip empty keys or keys equal to 'paragraphs'
            if not key1 or key1 == 'paragraphs':
                continue

            # If the value is a dictionary, recursively search for similar keys
            if isinstance(value1, dict):
                similar_keys += compare_dicts([value1], similarity_threshold)
            # Recursively search for similar keys in the nested dictionaries
            #similar_keys += compare_dicts(current_dict[key1], similarity_threshold)

            # Iterate over the subsequent dictionaries
            for j in range(i + 1, len(dict_list)):
                # Get the subsequent dictionary
                subsequent_dict = dict_list[j]

                # Iterate over the keys in the subsequent dictionary
                for key2 in subsequent_dict:
                    # Skip empty keys or keys equal to 'paragraphs'
                    if not key2 or key2 == 'paragraphs':
                        continue

                    # Calculate the similarity between the current key and the subsequent key
                    similarity = documentSimilarity(key1, key2)

                    # If the similarity is below the threshold, add the current key to the list of similar keys
                    if similarity < similarity_threshold:
                        similar_keys.append(key1)

    # Return the list of similar keys
    return similar_keys


compare_dicts(list_of_dicts, 2)

['RF Wireless World',
 'RF Wireless World',
 'RF Wireless World',
 'RF Wireless World',
 'RF Wireless World',
 'RF Wireless World',
 'RF Wireless World',
 'A BPSK Modulator for 2GHz to 12GHz',
 'A BPSK Modulator for 2GHz to 12GHz',
 'A BPSK Modulator for 2GHz to 12GHz',
 'A BPSK Modulator for 2GHz to 12GHz',
 'A BPSK Modulator for 2GHz to 12GHz',
 'A BPSK Modulator for 2GHz to 12GHz',
 'Digital Phase Modulation: BPSK, QPSK, DQPSK',
 'Digital Phase Modulation: BPSK, QPSK, DQPSK',
 'Digital Phase Modulation: BPSK, QPSK, DQPSK',
 'Digital Phase Modulation: BPSK, QPSK, DQPSK',
 'Digital Phase Modulation: BPSK, QPSK, DQPSK',
 'Not Acceptable!',
 'Not Acceptable!',
 'Not Acceptable!',
 'Not Acceptable!',
 'Phase-shift keying',
 'Phase-shift keying',
 'Phase-shift keying',
 'Phase Shift Keying – PSK',
 'Phase Shift Keying – PSK',
 'Digital Communication - Phase Shift Keying']

# New Algo

## Create Dataset

In [ ]:
import wikipediaapi
import json

def extract_headings_and_paragraphs(section, data, parent_key=None):
    """Recursively extract headings and paragraphs from a wikipediaapi.Section object."""
    key = section.title.lower()
    if parent_key:
        key = f"{parent_key}/{key}"
    data[key] = []
    for paragraph in section.text.split('\n'):
        if paragraph and paragraph[0] != '=':
            data[key].append(paragraph)
    for sub_section in section.sections:
        extract_headings_and_paragraphs(sub_section, data, key)
    return data

# Create a Wikipedia API object
wiki_api = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

# Specify the search term and retrieve the search results
search_term = 'Dip-coating'
wikipedia_page = wiki_api.page(search_term)

# Extract the headings and paragraphs of the page, starting from the first heading
data = {}
for section in wikipedia_page.sections:
    data = extract_headings_and_paragraphs(section, data)

# Save the data to a JSON file
with open('data.json', 'w') as f:
    json.dump(data, f)


In [ ]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/Wikipedia:WikiProject_Science/Popular_pages"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})
#print(indiatable)
df=pd.read_html(str(indiatable))
# convert list to dataframe
df=pd.DataFrame(df[0])
search_terms = df['Page title'].head(500).tolist()
#print(search_terms)

In [ ]:
import wikipediaapi
import json

def extract_headings_and_paragraphs(section, data, parent_key=None):
    """Recursively extract headings and paragraphs from a wikipediaapi.Section object."""
    key = section.title.lower()
    if parent_key:
        key = f"{parent_key}/{key}"
    data[key] = []
    for paragraph in section.text.split('\n'):
        if paragraph and paragraph[0] != '=':
            data[key].append(paragraph)
    for sub_section in section.sections:
        extract_headings_and_paragraphs(sub_section, data, key)
    return data

# Create a Wikipedia API object
wiki_api = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

# Iterate over the search terms
for search_term in search_terms:
    # Retrieve the Wikipedia page for the search term
    wikipedia_page = wiki_api.page(search_term)

    # Extract the headings and paragraphs of the page, starting from the first heading
    data = {}
    for section in wikipedia_page.sections:
        data = extract_headings_and_paragraphs(section, data)

        # Save the data to a JSON file
    with open(f'dataset/{search_term}.json', 'w') as f:
        json.dump(data, f)

# DEFINITIONs

In [23]:
import wikipedia

word = "Gyaan Setu"
wikipedia.set_lang("en")

try:
    page = wikipedia.page(word)
    summary = page.summary.split("\n")
    for point in summary:
        print("- " + point)
except wikipedia.exceptions.PageError:
    print("Page not found")


Page not found


In [13]:
from nltk.corpus import wordnet

word = "midtown"
synsets = wordnet.synsets(word)

related_nouns = []

# Extract all the lemmas of each synset
for synset in synsets:
    for lemma in synset.lemmas():
        if lemma.synset().pos() == 'n':
            related_nouns.append(lemma.name())

# Print the related nouns
print("Related nouns for " + word + ":", related_nouns)


Related nouns for midtown: []


## Extract relevant info from the dataset

In [18]:
import json

# Open the JSON file
with open('dataset/Artificial intelligence.json', 'r') as f:
  # Load the data from the file
  data = json.load(f)

# Use the data
# print(data)


In [19]:
import csv

def save_to_csv(data, encoding='utf-8', errors='strict'):
  def encode(input, encoding='utf-8', errors='strict'):
    if isinstance(input, list):
      return input
    else:
      return input


  with open('output.csv', 'w', newline='', encoding=encoding) as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['headings', 'passage'])
    for key, value in data.items():
      headings = []
      while "/" in key:
        heading, key = key.split("/", 1)
        headings.append(heading)
      headings.append(key)
      passage = " ".join(value)
      writer.writerow([encode('/'.join(headings)), encode(passage)])


save_to_csv(data)


In [2]:
import json
import os
import csv

def save_to_csv(folder_path, encoding='utf-8', errors='strict'):
  def encode(input, encoding='utf-8', errors='strict'):
    if isinstance(input, list):
      return input
    else:
      return input

  # Open the output CSV file
  with open('output.csv', 'w', newline='', encoding=encoding) as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['headings', 'passage'])
    # loop over all files in the folder
    for filename in os.listdir(folder_path):
        # check if file is json file
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path) as json_file:
                data = json.load(json_file)
                for key, value in data.items():
                    headings = []
                    while "/" in key:
                        heading, key = key.split("/", 1)
                        headings.append(heading)
                    headings.append(key)
                    passage = " ".join(value)
                    writer.writerow([encode('/'.join(headings)), encode(passage)])
    print("Data from all json files are successfully added to output.csv file")

save_to_csv("dataset")

Data from all json files are successfully added to output.csv file


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from io import StringIO

# Load the dataset into a pandas dataframe
df = pd.read_csv('output.csv')

df


,headings,passage
0,cause,This earthquake was caused by movement on a NW...
1,historical context,"Earthquakes mark the history of L'Aquila, a ci..."
2,effects,"The earthquake caused damage to between 3,000 ..."
3,effects/homeless camps,"Around 40,000 people who were made homeless by..."
4,effects/building standards,Poor building standards or construction materi...
...,...,...
7394,see also,Amateur exoplanet discoveries 9Spitch AWI0005x...
7395,references,Media related to Zooniverse at Wikimedia Commons
7396,time,ΔT (timekeeping) the difference between two ti...
7397,temperature,to mean a difference in temperature including ...


In [6]:
# Reset the index and drop any rows with missing values
df = df.reset_index(drop=True).dropna()

df


,headings,passage
0,cause,This earthquake was caused by movement on a NW...
1,historical context,"Earthquakes mark the history of L'Aquila, a ci..."
2,effects,"The earthquake caused damage to between 3,000 ..."
3,effects/homeless camps,"Around 40,000 people who were made homeless by..."
4,effects/building standards,Poor building standards or construction materi...
...,...,...
6185,see also,Amateur exoplanet discoveries 9Spitch AWI0005x...
6186,references,Media related to Zooniverse at Wikimedia Commons
6187,time,ΔT (timekeeping) the difference between two ti...
6188,temperature,to mean a difference in temperature including ...


In [7]:
# Create the dataset
dataset = []
for i in range(len(df)):
    dataset.append([df['headings'][i], df['passage'][i]])

#print(dataset)

In [8]:
import spacy

# The dataset you want to train the model on
""" dataset = [
    ["history", "Artificial beings with intelligence appeared as storytelling devices in antiquity, and have been common in fiction, as in Mary Shelley's Frankenstein or Karel Čapek's R.U.R."],
    ["goals", "The general problem of simulating (or creating) intelligence has been broken down into sub-problems. These consist of particular traits or capabilities that researchers expect an intelligent system to display."],
    ["goals/reasoning, problem-solving", "The general problem of simulating (or creating) intelligence has been broken down into sub-problems. These consist of particular traits or capabilities that researchers expect an intelligent system to display. The traits described below have received the most attention."],
    ["goals/knowledge representation", "Early researchers developed algorithms for searching and for solving problems that are now central to the field of artificial intelligence (AI)."]
] """

' dataset = [\n    ["history", "Artificial beings with intelligence appeared as storytelling devices in antiquity, and have been common in fiction, as in Mary Shelley\'s Frankenstein or Karel Čapek\'s R.U.R."],\n    ["goals", "The general problem of simulating (or creating) intelligence has been broken down into sub-problems. These consist of particular traits or capabilities that researchers expect an intelligent system to display."],\n    ["goals/reasoning, problem-solving", "The general problem of simulating (or creating) intelligence has been broken down into sub-problems. These consist of particular traits or capabilities that researchers expect an intelligent system to display. The traits described below have received the most attention."],\n    ["goals/knowledge representation", "Early researchers developed algorithms for searching and for solving problems that are now central to the field of artificial intelligence (AI)."]\n] '

In [9]:
import nltk
from sklearn.model_selection import train_test_split

# Preprocess the dataset
X = []  # List of passages
y = []  # List of labels

for label, passage in dataset:
    doc = nltk.word_tokenize(passage)
    fdist = nltk.FreqDist(doc)
    X.append(fdist)
    y.append(label)




In [6]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train a classifier using the training data
classifier = nltk.NaiveBayesClassifier.train(zip(X_train, y_train))

# Test the classifier on the test data
print(nltk.classify.accuracy(classifier, zip(X_test, y_test)))


KeyboardInterrupt: 

In [12]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook

# Split the dataset into chunks of data
chunk_size = 500
X_chunks = [X[i:i+chunk_size] for i in range(0, len(X), chunk_size)]
y_chunks = [y[i:i+chunk_size] for i in range(0, len(y), chunk_size)]

X_train, X_test, y_train, y_test = train_test_split(X[:chunk_size], y[:chunk_size], test_size=0.2)
classifier = nltk.NaiveBayesClassifier.train(zip(X_train, y_train))
for X_chunk, y_chunk in tqdm_notebook(zip(X_chunks[1:], y_chunks[1:]),total=len(X_chunks[1:])):
    X_train = X_train + X_chunk
    y_train = y_train + y_chunk
    classifier = nltk.NaiveBayesClassifier.train(zip(X_train, y_train))

    # Test the classifier on the test data
    print(nltk.classify.accuracy(classifier, zip(X_test, y_test)))


C:\Users\sukri\AppData\Local\Temp/ipykernel_12592/1220834192.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for X_chunk, y_chunk in tqdm_notebook(zip(X_chunks[1:], y_chunks[1:]),total=len(X_chunks[1:])):


  0%|          | 0/12 [00:00<?, ?it/s]

0


In [ ]:

# Use the classifier to predict the heading for a new paragraph
paragraph = "what is the need for artificial intelligence "
tokens = nltk.word_tokenize(paragraph)
fdist = nltk.FreqDist(tokens)
prediction = classifier.classify(fdist)
print(f"Predicted heading: {prediction}")

## Formulas

In [2]:
import json
import os
from pprint import pprint
import requests

subscriptionKey = 'b42a0d68f0c949f582f44edddd782914'
endpoint = "https://api.bing.microsoft.com/v7.0/images/search/"

query = "Laplace transform"

mkt = 'en-US'
params = {'q': query, 'mkt': mkt}
headers = {'Ocp-Apim-Subscription-Key': subscriptionKey}

try:
    response = requests.get(endpoint, headers=headers, params=params)
    response.raise_for_status()

    data = response.json()

    images = data["value"]
    for image in images:
        print("Content URL: ", image["contentUrl"])
        print("Thumbnail URL: ", image["thumbnailUrl"])
        print("Alt text: ", image["name"])
except requests.exceptions.HTTPError as err:
    print (f"HTTP Error Occured: {err}")


Content URL:  https://d2vlcm61l7u1fs.cloudfront.net/media/bbc/bbcc8f2a-7972-49a0-8fc6-56f9644dcc23/phpB5zv1R.png
Thumbnail URL:  https://tse1.mm.bing.net/th?id=OIP.lSA9vyArHc9L7hfxD15U1wHaI5&pid=Api
Alt text:  Solved: Use Laplace Transforms To Solve The Initial Value ... | Chegg.com
Content URL:  https://miro.medium.com/max/1276/1*A-JtLVxLP6P-xpnF4B_ulA.jpeg
Thumbnail URL:  https://tse4.mm.bing.net/th?id=OIP.iQqcCRNC9C9RREu5nh3JjAHaJl&pid=Api
Alt text:  Laplace Transforms: An Untold Story | by Kailash Nagarajan | Medium
Content URL:  https://www.coursehero.com/qa/attachment/13316238/
Thumbnail URL:  https://tse1.mm.bing.net/th?id=OIP.sjC3o16hMcF37gQ0-ySqNAHaGr&pid=Api
Alt text:  [Solved] The Laplace transform of the function, whose graph is the ...
Content URL:  http://media.cheggcdn.com/media%2F7f0%2F7f0f6e98-53d1-4aa7-bb2c-1e554d75dd7a%2FphpSS5ot8.png
Thumbnail URL:  https://tse3.mm.bing.net/th?id=OIP.VmNE0sEZGXsq3wthnfmsJgHaGv&pid=Api
Alt text:  Use Laplace Transforms To Solve The I

In [5]:
import requests
from bs4 import BeautifulSoup
import wikipediaapi

wiki = wikipediaapi.Wikipedia(
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)

search_query = "laplace transform"
page = wiki.page(search_query)

if page.exists():
    # Make a request to the Wikipedia page
    query = page.title
    url = f"https://en.wikipedia.org/wiki/{query}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract the equations and their descriptions from the page
    equations = {}
    for img in soup.find_all("img", class_="mwe-math-fallback-image-inline"):
        eq_src = img["src"]
        print(eq_src)
    if not equations:
        for img in soup.find_all("img", class_="mwe-math-fallback-image-display"):
            eq_src = img["src"]
            print(eq_src)
    print(equations)
else:
    print("Page not found.")

https://wikimedia.org/api/rest_v1/media/math/render/svg/65658b7b223af9e1acc877d848888ecdb4466560
https://wikimedia.org/api/rest_v1/media/math/render/svg/01d131dfd7673938b947072a13a9744fe997e632
https://wikimedia.org/api/rest_v1/media/math/render/svg/b49caba366b94ee1ecb91eee31d5a709f8b0beaa
https://wikimedia.org/api/rest_v1/media/math/render/svg/b2734e92c352d7115a47553097a0e35462f915ef
https://wikimedia.org/api/rest_v1/media/math/render/svg/b3afb2261b1eddec57e0578ed13c92cb3a891821
https://wikimedia.org/api/rest_v1/media/math/render/svg/2fcaa5193838b9f415e2422da9e9880de78ecc88
https://wikimedia.org/api/rest_v1/media/math/render/svg/f162776eba3f1c5666aa134b0dd270b7ce62a998
https://wikimedia.org/api/rest_v1/media/math/render/svg/545fd099af8541605f7ee55f08225526be88ce57
https://wikimedia.org/api/rest_v1/media/math/render/svg/0c16629cf23074434b3ed8d76c8408dc3de0c93e
https://wikimedia.org/api/rest_v1/media/math/render/svg/6c439e5ee4a098d215f3da4140f07fe1cc1c8a38
https://wikimedia.org/api/rest

In [8]:
equations = {}
for img in soup.find_all("img", class_="mwe-math-fallback-image-inline"):
    eq_src = img["src"]
    for p in soup.find_all("p")[::-1]:
        if p.find("img") != None and p.find("img") == img:
            continue
        if p.next_sibling.name == "img" and p.next_sibling == img:
            eq_description = p.text
            break
    equations[eq_src] = eq_description
    print(eq_src, eq_description)
if not equations:
    for img in soup.find_all("img", class_="mwe-math-fallback-image-display"):
        eq_src = img["src"]
        for p in soup.find_all("p")[::-1]:
            if p.find("img") != None and p.find("img") == img:
                continue
            if p.next_sibling.name == "img" and p.next_sibling == img:
                eq_description = p.text
                break
        equations[eq_src] = eq_description
        print(eq_src, eq_description)
print(equations)            


https://wikimedia.org/api/rest_v1/media/math/render/svg/65658b7b223af9e1acc877d848888ecdb4466560 In statistical mechanics, the Laplace transform of the density of states 



g
(
E
)


{\displaystyle g(E)}

 defines the partition function.[35] That is, the canonical partition function 



Z
(
β
)


{\displaystyle Z(\beta )}

 is given by




Z
(
β
)
=

∫

0


∞



e

−
β
E


g
(
E
)

d
E


{\displaystyle Z(\beta )=\int _{0}^{\infty }e^{-\beta E}g(E)\,dE}


and the inverse is given by




g
(
E
)
=


1

2
π
i




∫


β

0


−
i
∞



β

0


+
i
∞



e

β
E


Z
(
β
)

d
β


{\displaystyle g(E)={\frac {1}{2\pi i}}\int _{\beta _{0}-i\infty }^{\beta _{0}+i\infty }e^{\beta E}Z(\beta )\,d\beta }



https://wikimedia.org/api/rest_v1/media/math/render/svg/01d131dfd7673938b947072a13a9744fe997e632 In statistical mechanics, the Laplace transform of the density of states 



g
(
E
)


{\displaystyle g(E)}

 defines the partition function.[35] That is, the canonical partition function 



Z
(
β
)


{\

In [4]:
import textstat

text = "Cloud computing and associated solutions provide access through the web to computing resources and products, including development tools, business applications, compute services, data storage, and networking solutions. These cloud services are hosted at a software vendor’s data center and managed by the cloud services provider or onsite at a customer's data center."

print(textstat.reading_time(text)/60)


0.07733333333333332
